# Chapter 5. 오차역전파법 (Backpropagation)  

* 수치 미분의 장점과 단점
    * 장점 - 단순하고 구현이 간단.
    * 단점 - 시간이 오래 걸림.
        * 가중치 매개변수의 기울기를 효율적으로 계산하기 위해 Backpropagation 도입.


* Backpropagation의 이해
    * 수식 - 설명 안함.
    * 계산 그래프(시각적) - CS231n 참고

## 5.1 계산 그래프 (Computational Graph)  

* 계산 그래프 - 계산 과정을 그래프로 표현. (TensorFlow와 비슷한 접근).
    * 노드(Node) - 원(O)으로 표현 - 연산
    * 에지(Edge) - 화살표 - 연산의 흐름

### 5.1.1 계산 그래프로 풀다  

*문제 1 : 현빈 군은 슈퍼에서 1개에 100원인 사과 2개를 샀습니다. 이때 지불금액을 구하세요. 단 소비세가 10% 부과됩니다.*  
(문제 1은 곱으로만 표현 가능)  

<center>**Figure 5-1 계산 그래프로 풀어본 문제 1의 답**</center>
![Figure_5-01](./images/Figure_5-01.png)  

**Figure 5-1**은 상수와 연산이 완전히 분리되어 있지 않으므로 상수와 연산자를 완전히 분리하면 **Figure 5-2**와 같이 표현됨.  
(이후에는 **Figure 5-2**와 같은 형식으로 표현)  

<center>**Figure 5-2 계산 그래프로 풀어본 문제 1의 답 : '사과의 개수'와 '소비세'를 변수로 취급해 원 밖에 표기**</center>
![Figure_5-02](./images/Figure_5-02.png)

*문제 2 : 현빈 군은 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개 150원입니다. 소비자세가 10%일 때 지불 금액을 구하세요.*  
(문제 2는 곱과 합으로 표현 가능)

<center>**Figure 5-3 계산 그래프로 풀어본 문제2의 답**</center>
![Figure_5-03](./images/Figure_5-03.png)

* 계산 그래프를 이용한 문제풀이의 흐름
    1. 계산 그래프를 구성한다.
    2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.

2와 같이 왼쪽에서 오른쪽으로 진행하는 단계를 순전파 (Forward Propagation).  
반대로 오른쪽에서 왼쪽으로 진행하는 단계를 역전파 (Backpropagation).

### 5.1.2 국소적 계산  

계산 그래프의 특징은 **국소적 계산**을 전파함으로써 최종 결과를 얻는 것.  
**국소적이란** 자신과 직접 관계된 작은 범위.  
(전체와는 상관없이 자신과 관계된 정보만으로 다음 결과를 출력할 수 있음.)

<center>**Figure 5-4 사과 2개를 포함해 여러 식품을 구입하는 예**</center>  
![Figure_5-04](./images/Figure_5-04.png)

**Figure 5-4**에서 여러 식품을 구입하여 총금액이 4,000원.  
이후 사과를 추가로 더 구매한다면 두 값을 더하는 계산(4,000 + 200 -> 4,200)은 4,000이라는 숫자가 어떻게 계산되어 나왔느냐와 관계가 없음.  
결국 각 노드는 자신과 관련한 계산(Figure 5-4에서는 두 숫자의 덧셈) 외에는 신경쓸 사항이 없음.  


위에서 처럼 계산 그래프는 국소적 계산에만 집중하면 됨.  
**간단한 국소적인 계산의 조합으로 복잡한 계산 결과를 얻을 수 있음.**

### 5.1.3 왜 계산 그래프로 푸는가?  

* 계산 그래프의 이점
    * 국소적 계산
        * 각 노드의 단순한 계산으로 문제를 단순화 할 수 있음.
        * 각 노드의 중간 계산 결과를 모두 보관 가능.
    * **역전파를 통해 *미분*을 효율적으로 계산 가능.**


역전파 설명을 위해 문제 1로 다시 돌아감.  
*문제 1 : 현빈 군은 슈퍼에서 1개에 100원인 사과 2개를 샀습니다. 이때 지불금액을 구하세요. 단 소비세가 10% 부과됩니다.*  

*문제 1-1 : 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지 알고 싶음.*  
위 문제는 **사과 가격에 대한 지불 금액의 미분**을 구하는 문제임.

사과 값을 $x$, 지불 금액을 $L$이라 했을 때 $\frac{\partial L}{\partial x}$을 구하는 문제.  
(사과 값이 **아주 조금** 올랐을 때 지불 금액이 얼마나 증가하느냐?)

**사과 가격에 대한 지불 금액의 미분**은 역전파를 하면 구할 수 있음.  
**Figure 5-5**는 계산 그래프 상의 역전파를 미분을 통혜서 구해 놓은 결과.(자세한 내용은 뒤에서)  

<center>**Figure 5-5 역전파에 의한 미분 값의 전달**</center>
![Figure_5-05](./images/Figure_5-05.png)

역전파는 순전파와는 반대 방향의 화살표(붉은 선)으로 표시.  
이 전파는 **국소적 미분**을 전달하고 그 미분 값을 화살표의 아래에 표시.  
이 예에서 역전파는 오른쪽에서 왼쪽으로 **1 -> 1.1 -> 2.2** 순으로 미분 값을 전달.  
이 결과로부터 **사과 가격에 대한 지불 금액의 미분**값은 2.2라는 것을 알 수 있음.  
사과가 1원 오르면 최종금액은 2.2원 오름.  
(사과 가격이 $x$만큼 오르면 최종금액은 2.2$x$만큼 오름.)  


## 5.2 연쇄법칙 (Chain Rule)  

**Figure 5-5**처럼 오른쪽에서 왼쪽으로 **국소적 미분**을 전달하는 원리는 **연쇄법칙(Chain Rule)**에 따른 것.  
이 절에서 연쇄법칙을 설명하고 그것이 그래프 상의 역전파와 같다는 사실을 확인.

### 5.2.1 계산 그래프의 역전파  

**Figure 5-6**에서 $y=f(x)$라는 계산의 역전파를 표현.  

<center>**Figure 5-6 계산 그래프의 역전파 : 순방향과는 반대 방향으로 국소적 미분을 곱한다.**</center>  
![Figure_5-06](./images/Figure_5-06.png)

**Figure 5-6**과 같이 역전파의 계산 절차는 신호 $E$에 노드의 국소적 미분$(\frac{\partial y}{\partial x})$를 곱한 후 다음 노드로 전달하는 것.  
1. **국소적 미분**은 순전파 때의  $y=f(x)$ 계산의 미분을 구한다는 것이며, 이는 $x$에 대한 $y$의 미분$(\frac{\partial y}{\partial x})$을 구한다는 뜻.  
(예 : $y=f(x)=x^2$이라면 $(\frac{\partial y}{\partial x}) = 2x$)
2. 그리고 이 국소적인 미분을 상류에서 전달된 값(이 예에서는 E)에 곱해 앞으로 전달하는 것.  

위의 방식을 따르면 목표로 하는 미분 값을 효율적으로 구할 수 있다는 것이 역전파의 핵심.

### 5.2.2 연쇄법칙이란?  

**합성 함수** - 여려 함수로 구성된 함수.  

예) $x = {(x + y)}^2$는 **Equation 5.1**처럼 두 개의 식으로 구성.  
$$ 
\mathbf{Equation \ 5.1} \\ 
z = t^2 \\ 
t = x + y \\
$$

* 연쇄법칙
    * *합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.*  

**Equation 5.1**을 예로 설명, $\frac{\partial z}{\partial x}$ ($x$에 대한 $z$의 미분)은  $\frac{\partial z}{\partial t}$  ($t$에 대한 $z$의 미분)과  $\frac{\partial t}{\partial x}$($x$에 대한 $t$의 미분)의 곱으로 나타낼 수 있음.  
수식으로는 **Equation 5.2**처럼 표기.  

$$
\mathbf{Equation \ 5.2} \\
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} 
$$  

**Equation 5.2**는 다음과 같이 $\partial t$를 서로 지울 수 있음.
$$
\require{cancel}
\frac{\partial z}{\partial x} = \frac{\partial z}{\cancel{\partial t}} \frac{\cancel{\partial t}}{\partial x}
$$  

연쇄 법칙을 써서 **Equation 5.2**의 미분 $\frac{\partial z}{\partial x}$를 진행. 먼저 **Equation 5.1**의 국소적 미분(편미분)을 구함.  
$$ 
\mathbf{Equation \ 5.3} \\
\frac{\partial z}{\partial t} = 2t \\ 
\frac{\partial t}{\partial x} = 1
$$  

**Equation 5.3**과 같이 $\frac{\partial z}{\partial t}$는 $2t$이고, $\frac{\partial t}{\partial x}$는 1.(미분 공식에서 해석적으로 구한 결과)  
최종적으로 구하고자 하는 $\frac{\partial z}{\partial x}$는 **Equation 5.3**에서 구한 두 미분을 곱해서 계산.  

$$
\mathbf{Equation \ 5.4} \\
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t \centerdot 1 = 2(x + y)
$$  

### 5.2.3 연쇄법칙과 계산 그래프  

**Equation 5.4**의 연쇄법칙 계산을 계산 그래프로 표현하면 **Figure 5-7**처럼 표현 가능.  
(2제곱 계산은 '**$\ast\ast2$**'노드로 표현)  

<center>**Figure 5-7 *Equation 5.4*의 계산 그래프 : 순전파와는 반대 방향으로 국소적 미분을 곱하여 전달한다.**</center>
![Figure_5-07](./images/Figure_5-07.png)  

**Figure 5-7**과 같이 계산 그래프의 역전파는 오른쪽에서 왼쪽으로 신호를 전파.  
역전파의 계산 절차에서는 노드로 들어온 입력 신호에 그 노드의 국소적 미분(편미분)을 곱한 후 다음 노드로 전달.  

1. 역전파의 첫 신호인 $\frac{\partial z}{\partial z}$는 1.
2. '$\ast\ast2$'노드에서는 입력은 $\frac{\partial z}{\partial z}$에 국소적인 미분 $\frac{\partial z}{\partial t}$를 곱하고 다음 노드로 전달.
    * 순전파시 입력이 t이고 출력이 z이므로 이 노드의 (국소적) 미분은 $\frac{\partial z}{\partial t}$.
3. +노드에서는 입력이 $\frac{\partial z}{\partial z} \frac{\partial z}{\partial t}$이고 국소적인 미분 $\frac{\partial t}{\partial x}$를 곱하여 다음 노드로 전달하면 $\frac{\partial z}{\partial z} \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}$가 됨.
    * $\frac{\partial z}{\partial z} \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = \frac{\partial z}{\cancel{\partial z}} \frac{\cancel{\partial z}}{\cancel{\partial t}} \frac{\cancel{\partial t}}{\partial x} = \frac{\partial z}{\partial x}$가 성립되어 **$x$에 대한 $z$의 미분**이 됨.(즉, 역전파가 하는 일은 연쇄 법칙의 원리와 같음.)

<center>**Figure 5-8 계산 그래프의 역전파 결과에 따르면 $\frac{\partial z}{\partial x}$는 $2(x + y)$가 된다.**</center>
![Figure_5-08](./images/Figure_5-08.png)  

### 5.3.1 덧셈 노드의 역전파  

식 $z = x + y$를 대상으로 역전파 설명.  

$z = x + y$의 편미분은 **Equation 5.5**와 같음.

$$
\mathbf{Equation \ 5.5} \\  
\frac{\partial z}{\partial x} = 1 \\  
\frac{\partial z}{\partial y} = 1
$$  

**Equation 5.5**를 **Figure 5-9**와 같이 나타낼 수 있음.

<center>**Figure 5-9 덧셈 노드의 역전파 : 왼쪽이 순전파. 오른쪽이 역전파다. 덧셈 노드의 역전파는 입력 값을 그대로 흘려보낸다.**</center>
![Figure_5-09](./images/Figure_5-09.png)  

**Figure 5-9**와 같이 덧셈 노드의 역전파는 1을 곱하기만 할 뿐이므로 입력된 값을 그대로 다음 노드로 전달.  

**Figure 5-9**에서는 상류에서 전해진 미분 값을 $\frac{\partial L}{\partial z}$라고 했는데, 이는 **Figure 5-10**에서와 같이 최종적으로 L이라는 값을 출력하는 큰 계산 그래프를 가정하기 때문.  
$z = x + y$ 계산은 그 근 계산 그래프의 중간 어딘가에 존재하고, 상류로부터 $\frac{\partial L}{\partial z}$값이 전해저 내려온 것이고 다시 하류로 $\frac{\partial L}{\partial x} (= \frac{\partial L}{\partial z} \frac{\partial z}{\partial x} )$ 과 $\frac{\partial L}{\partial y} (= \frac{\partial L}{\partial z} \frac{\partial z}{\partial y})$ 값을 전달 하는 것임.  

<center>**Figure 5-10 최종 출력으로 가는 계산의 중간에 덧셈 노드가 존재한다. 역전파에서는 국소적 미분이 가장 오른쪽의 출력에서  
시작하여 노드를 타고 역방향(왼쪽)으로 전파된다.**</center>
![Figure_5-10](./images/Figure_5-10.png)  

**Figure 5-11**에서는 **$10 + 5 = 15$**라는 순전파에 대한 식이 주어 지고, 상류에서 1.3이라는 값이 전해진다고 가정함.  
덧셈의 역전파는 입력 신호를 다음노드로 그대로 출력할 뿐이므로 1.3을 다음노드로 전달함.  

<center>**Figure 5-11 덧셈노드 역전파의 구체적인 예**</center>
![Figure_5-11](./images/Figure_5-11.png)  

### 5.3.2 곱셈 노드의 역전파  

식 $z = xy$라는 식으로 역전파 설명.  

이 식의 미분은 **Equation 5.6**과 같음.  

$$
\mathbf{Equation \ 5.6} \\  
\frac{\partial z}{\partial x} = y \\  
\frac{\partial z}{\partial y} = x
$$  

**Equation 5.6**을 계산 그래프로 표현하면 **Figure 5-12**와 같음.  

<center>**Figure 5-12 곱셈 노드의 역전파 : 왼쪽이 순전파. 오른쪽이 역전파다.**</center>
![Figure_5-12](./images/Figure_5-12.png)  

**Figure  5-12**와 같이 곱셈 노드 역전파는 상류 값에 순전파 때의 입력 신호들을 **서로 바꾼 값**을 곱해서 하류로 전달.  

**Figure 5-13**에서는 **$10 \times 5 = 50$**이라는 순전파에 대한 식이 주어지고, 역전파 때 상류에서 1.3이 전해진다고 가정함.  
곱셈의 역전파는 입력 신호를 바꾼 값을 곱하여 전달하므로 10이 전해졌던 쪽으로는 $6.5(= 1.3 \times 5)$이 흐르고, 5가 전해졌던 쪽으로는 $13 (= 1.3 \times 10)$이 흐름.  

<center>**Figure 5-13 곱셈 노드 역전파의 구체적인 예**</center>
![Figure_5-13](./images/Figure_5-13.png)  

**덧셈의 역전파는 순방향의 신호 값을 저장할 필요가 없으나, 곱셈의 역전파는 순방향의 신호 값을 저장할 필요가 있음.**

### 5.3.3 사과 쇼핑의 예  

* 사과 가격에 대한 지불 금액의 미분 : 2.2
* 사과 개수에 대한 지불 금액의 미분 : 110
* 소비세에 대한 지불 금액의 미분    : 1.1
    * 단위가 다르므로 주의.

<center>**Figure 5-14 사과 쇼핑의 역전파 예**</center>
![Figure_5-14](./images/Figure_5-14.png)

<center>**Figure 5-15 사과와 귤 쇼핑의 역전파 예 : 빈 상자 안에 적절한 숫자를 넣어 역전파를 완성하시오.**</center>
![Figure_5-15](./images/Figure_5-15.png)

## 5.4 단순한 계층 구현하기  

* 구현
    * 곱셈 노드(MulLayer)
    * 덧셈 노드(AddLayer)

* Layer 구현의 공통점
    * forward() method : forward propagation
    * backward() method : backward propagation

### 5.4.1 곱셈 계층  

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out

    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

\_\_init\_\_()에서는 인스턴스 변수 x와 y를 초기화.
이 변수들은 순전파 시의 입력 값을 유지하기 위해 필요.
backward() 실행시 상류에서 넘어온 미분(dout)에 forward()시의 입력 값을 **서로 바꿔** 곱한 후 하류로 전달.

<center>**Figure 5-16 사과 2개 구입**</center>
![Figure_5-16](./images/Figure_5-16.png)

곱셈 계층으로 구현된 class MulLayer를 이용하여 순전파(forward())를 수행시켜 볼 수 있음.

**Figure 5-16**을 python으로 구현하면 다음과 같음.

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print("Price : ", int(price)) # 220

Price :  220


각 변수에 대한 미분은 역전파(backward())를 통해 구할 수 있음.

In [3]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

# print(dapple, dapple_num, dtax) # 2.2 110 200
print("dApple : ", dapple)
print("dApple_num : ", int(dapple_num))
print("dTax : ", dtax)

dApple :  2.2
dApple_num :  110
dTax :  200


backward() 호출 순서는 forward() 때와는 반대 순서.  
backward()가 전달 받는 인수는 **순전파의 추력에 대한 미분**임.(이 책에서는 미분값을 저장하는 변수에 **d**를 붙여 구분해 줌.)

### 5.4.2 덧셈 계층  

In [4]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

덧셈 계층에서는 역전파시 순전파시의 인수를 사용하지 않으므로, \_\_init\_\_()에서는 아무 일도 하지 않습니다.(**pass**)  

* 순전파(forward propagation)
    * 입력받은 두 인수를 더해서 전달.
* 역전파(backward propagation)
    * 상류에서 전달 받은 미분(dout)을 그대로 하류로 전달.

<center>**Figure 5-17 사과 2개와 귤 3개 구입**</center>
![Figure_5-17](./images/Figure_5-17.png)  

**Figure 5-17**을 python으로 구현하면 다음과 같음.

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# Layers
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# Forward Propagration
apple_price = mul_apple_layer.forward(apple, apple_num) # (1)
orange_price = mul_orange_layer.forward(orange, orange_num) # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) # (3)
price = mul_tax_layer.forward(all_price, tax) # (4)

print("Price : ", int(price))

# Backward Propagation
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # (3)
dorange, dorange_num = mul_orange_layer.backward(dapple_price) # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # (1)

# print(price) # 715
# print(dapple_num, dapple, dorange, dorange_num, dtax) # 110 2.2 3.3 165 650
print("dApple_num : ", int(dapple_num))
print("dApple : ", dapple)
print("dOrange : ", round(dorange, 1))
print("dOrange_num : ", int(dorange_num))
print("dTax : ", dtax)

Price :  715
dApple_num :  110
dApple :  2.2
dOrange :  3.3
dOrange_num :  165
dTax :  650


## 5.5 활성화 함수 계층 구현하기  

* 활성화 함수 (Activation Function)
    * ReLU (Rectified Linear Unit)
    * Sigmoid

### 5.5.1 ReLU 계층  

활성화 함수로 사용되는 ReLU는 **Equation 5.7**과 같이 표현.

$$
\mathbf{Equation \ 5.7} \\
y = 
\begin{cases}
x & (x > 0) \\
0 & (x \le 0)
\end{cases}
$$  

**Equation 5.7**에서 $x$에 대한 $y$의 미분은 **Equation 5.8**처럼 구할 수 있음.  

$$
\mathbf{Equation \ 5.8} \\
\frac{\partial y}{\partial x} = 
\begin{cases}
1 & (x > 0) \\
0 & (x \le 0)
\end{cases}
$$  

**Equation 5.8**에서와 같이 순전파 때의 입력 $x$가 0보다 크면 역전파는 상류의 값을 그대로 하류로 전달.
반면, 순전파 때 이벼력 $x$가 0 이하면 역전파 때는 하류로 신호를 전달하지 않음.(0을 보냄)  

계산 그래프로는 **Figure 5-18**처럼 표현.  

<center>**Figure 5-18 ReLU 계층의 계산 그래프**</center>
![Figure_5-18](./images/Figure_5-18.png)  

In [6]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

Relu 클래스는 \_\_init\_\_()에 mask라는 인스턴스 변수를 가짐.  
mask는 Treu/False로 구성된 numpy array로, 순전파의 입력인 x의 원소 값이 0이하인 인덱스는 True, 그 외(0보다 큰 원소)는 False로 유지.  

In [7]:
import numpy as np

x = np.array( [[1.0, -0.5], [-2.0, 3.0]])
print(x, "\n")

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]] 

[[False  True]
 [ True False]]


**Figure 5-18**과 같이 순전파 때의 입력 값이 0이하면 역전파 때의 값은 0이 되어야 함.  
그래서, 순전파때 만들어 놓은 mask의 원소가 True인 원소는 dout을 0으로 변경.  

### 5.5.2 Sigmoid 계층  

활성화 함수로 사용되는 시그모이드 함수는 **Equation 5.9**와 같이 표현.

$$
\mathbf{Equation \ 5.9} \\
y = \frac{1}{1 + exp(-x)}
$$  

**Equation 5.9**를 계산 그래프로 그리면 **Figure 5-19**처럼 표현.  

<center>**Figure 5-19 Sigmoid 계층의 계산 그래프(순전파)**</center>
![Figure_5-19](./images/Figure_5-19.png)  

**Figure 5-19**에서는 '**$\times$**'와 '**$+$**'말고도 '**$exp$**' 와 '**$/$**' 노드가 새롭게 추가.  
'**$exp$**'노드는 $y = exp(x)$ 계산을 수행하고 '**$/$**'노드는 $y = \frac{1}{x}$ 계산을 수행.  
**Figure 5-19**와 같이 **Equation 5.9**의 계산은 국소적 계산의 전파로 이루어 짐.  

#### 1단계  

$/$노드, 즉 $y = \frac{1}{x}$을 미분하면 **Equation 5.10**과 같이 됨.

$$
\mathbf{Equation \ 5.10} \\
\begin{align} 
\frac{\partial y}{\partial x} & = -\frac{1}{x^2} \\ 
& = -y^2
\end{align} 
$$

**Equation 5.10**의 계산 그래프는 아래와 같음.  

![Figure_5-19-1](./images/Figure_5-19-1.png)  

#### 2단계  
'$+$'노드는 상류의 값을 그대로 하류로 보냄.  
![Figure_5-19-2](./images/Figure_5-19-2.png)  

#### 3단계  
'**$exp$**'노드는 $y = exp(x)$ 계산을 수행하고, 그 미분은 **Equation 5.11**과 같음.  
$$
\mathbf{Equation \ 5.11} \\
\frac{\partial y}{\partial x} = exp(x)
$$  
계산 그래프에서는 상류의 값에 순전파 때의 출력($exp(-x)$)을 곱해서 하류로 전달.  
![Figure_5-19-3](./images/Figure_5-19-3.png)  

#### 4단계  
'$\times$'노드는 순전파 때 값을 **서로 바꿔** 곱해서 전달.  
이 예에서는 $-1$을 곱함.  

<center>**Figure 5-20 Sigmoid 계층의 계산 그래프**</center>
![Figure_5-20](./images/Figure_5-20.png)  

**Figure 5-20**에서 보면 역전파의 최종 출력인 $\frac{\partial L}{\partial y} y^2 exp(-x)$값이 하류 노드로 전파됨.  
$\frac{\partial L}{\partial y} y^2 exp(-x)$는 순전파의 입력 $x$와 출력 $y$만으로 계산 가능함이 확인 됨.  
**Figure 5-20**의 계산 그래프의 중간 과정을 생략하여 **Figure 5-21**처럼 단순한 '$Sigmoid$'노드 하나로 대체 가능.  

<center>**Figure 5-21 Sigmoid 계층의 계산 그래프(간소화 버전)**</center>
![Figure_5-21](./images/Figure_5-21.png)  

**Figure 5-21**은 중간 계산을 생략할 수 있어 **Figure 5-20**에 비해 더 효율적인 계산임.  
'
또한, $\frac{\partial L}{\partial y} y^2 exp(-x)$는 다음처럼 정리해서 표현 가능.  

$$
\mathbf{Equation \ 5.12} \\
\begin{align} 
\frac{\partial L}{\partial y} y^2 exp(-x) & = \frac{\partial L}{\partial y} \frac{1}{(1 + exp(-x))^2} exp(-x) \\
& = \frac{\partial L}{\partial y} \frac{1}{1 + exp(-x)} \frac{exp(-x)}{1 + exp(-x)} \\
& = \frac{\partial L}{\partial y} y (1 - y)
\end{align} 
$$  

이처럼 $Sigmod$ 계층의 역전파는 순전파의 출력($y$)만으로 계산 가능함.  

<center>**Figure 5-22 Sigmoid 계층의 그래프 : 순전파의 출력 $y$만으로 역전파를 계산할 수 있다.**</center>
![Figure_5-22](./images/Figure_5-22.png)  

$Sigmoid$ 계층을 python으로 구현하면 아래와 같음.  

In [8]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

이 구현에서는 순전파의 출력을 인스턴스 변수 out에 보관했다가, 역전파 때 그 변수를 사용해 계산을 수행.  

## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층  

* 신경망의 순전파 때 수행하는 행렬의 내적은 기하학에서 **어파인 변환(Affine transfomation)**이라고 함.

In [9]:
X = np.random.rand(2)    # Input
W = np.random.rand(2, 3) # Weight
B = np.random.rand(3)    # Bias

print(X.shape) # (2,)
print(W.shape) # (2, 3)
print(B.shape) # (3,)

Y = np.dot(X, W) + B
print(Y)

(2,)
(2, 3)
(3,)
[ 0.79093848  0.76577485  0.89965323]


**X**, **W**, **B**는 형상이 (2,), (2, 3), (3,)인 다차원 배열.  
그러면 뉴런의 가중치 합은 **Y = np.dot(X, W) + B** 처럼 계산하여 **Y**를 활성화 함수로 변환하여 다음 층으로 전파하는 것이 신경망 순전파의 흐름.  
(이때 행렬의 내적 계산은 대응하는 차원의 원소 수를 일치시키는 것이 핵심.)  

<center>**Figure 5-23 행렬의 내적에서는 대응하는 차원의 원소 수를 일치시킨다.**</center>
![Figure_5-23](./images/Figure_5-23.png)  

내적을 계산 하는 노드를 '$dot$'이라 하면, **np.dot(X, W) + B** 계산은 **Figure 5-24**처럼 표현.  

<center>**Figure 5-24 Affine 계층의 계산 그래프 : 변수가 행렬임에 주의. 각 변수의 형상을 변수명 위에 표기했다.**</center>
![Figure_5-24](./images/Figure_5-24.png)  

**Figure 5-24**는 비교적 단순한 계산 그래프이지만, 단. **X**, **W**, **B**가 행렬(다차원 배열)이라는 점에 주의해야 함.  
**Figure 5-24**의 행렬을 사용한 역전파도 행렬의 원소마다 전개해 보면 스칼라값을 사용한 지금까지의 계산 그래프와 같은 순서로 생각할 수 있음.  
전개해 보면 **Equation 5.13**과 같이 표현.  

$$
\mathbf{Equation \ 5.13} \\
\frac{\partial L}{\partial \boldsymbol{X}} = \frac{\partial L}{\partial \boldsymbol{Y}} \centerdot \boldsymbol{W}^T \\
\frac{\partial L}{\partial \boldsymbol{W}} = \boldsymbol{X}^T \centerdot \frac{\partial L}{\partial \boldsymbol{Y}}
$$  

**Equation 5.13**에서의 $\boldsymbol{W}^T$의 $T$는 전치행렬을 뜻함.  
전치행렬은 $\boldsymbol{W}$의 $(i, j)$ 위치의 원소를  $(j, i)$ 위치로 바꾼 것을 뜻함.  
수식으로는 **Equation 5.14** 같이 표현. 

$$
\mathbf{Equation \ 5.14} \\
\begin{align}
& \boldsymbol{W} = \begin{pmatrix} w_{11} & w_{21} & w_{31} \\ w_{12} & w_{22} & w_{32} \end{pmatrix} \\
& \boldsymbol{W}^T = \begin{pmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \\ w_{31} & w_{32} \end{pmatrix}
\end{align}
$$  

**Equation 5.14**와 같이 $\boldsymbol{W}$의 형상이 $(2, 3)$이었다면 전치행렬 $\boldsymbol{W}^T$의 형상은 $(3, 2)$가 됨.  
**Eqaution 5.13**을 바탕으로 계산 그래프의 역전파를 구해 보면 **Figure 5-25**처럼 표현.  

<center>**Figure 5-25 Affine 계층의 역전파 : 변수가 다차원 배열임에 주의. 역전파에서의 변수 형상은 해당 변수명 아래에 표기했다.**</center>
![Figure_5-25](./images/Figure_5-25.png)  

**Figure 5-25**의 계산 그래프에서는 각 변수의 형상에 주의.  
$\boldsymbol{X}$와 $\frac{\partial L}{\partial \boldsymbol{X}}$은 같은 형상이고, $\boldsymbol{W}$와 $\frac{\partial L}{\partial \boldsymbol{W}}$도 같은 형상임.  
$\boldsymbol{X}$와 $\frac{\partial L}{\partial \boldsymbol{X}}$이 같다는 것은 **Equation 5.15**를 보면 명확해짐.  

$$
\mathbf{Equation \ 5.15} \\
\begin{align}
& \boldsymbol{X} = (x_0, x_1, \cdots, x_n) \\
& \frac{\partial z}{\partial \boldsymbol{X}} = \left( \frac{\partial L}{\partial x_0}, \frac{\partial L}{\partial x_1}, \cdots, \frac{\partial L}{\partial x_n}, \right)
\end{align}
$$  

$\frac{\partial L}{\partial \boldsymbol{Y}}$의 형상이 $(3,)$ $\boldsymbol{W}$의 형상이 $(2, 3)$일 때, $\frac{\partial L}{\partial \boldsymbol{X}}$의 형상이 $(2,)$가 되도록 하는 $\frac{\partial L}{\partial \boldsymbol{Y}}$ 과 $\boldsymbol{W}$의 내적은 **Figure 5-26**에서 확인할 수 있음.  
(**Equation 5.13** 참고)

<center>**Figure 5-26 행렬 내적('dot'노드)의 역전파는 행렬의 대응하는 차원의 원소 수가 일치하도록 내적을 조립하여 구할 수 있다.**</center>
![Figure_5-26](./images/Figure_5-26.png)  

### 5.6.2 배치용 Affine 계층  

이전의 Affine 계층은 입력 데이터로 $\boldsymbol{X}$ 하나만을 고려하였음.  
이제 배치용 Affine 계층을 고려.  
배치용 Affine 계층을 계산 그래프로 표현 하면 **Figure 5-27**과 같음.  

<center>**Figure 5-27 배치용 Affine 계층의 계산 그래프**</center>
![Figure_5-27](./images/Figure_5-27.png)    

이전과 다른 부분은 입력인 $\boldsymbol{X}$의 형상이 $(N, 2)$가 된 것 뿐임.  
역전파 때는 행렬의 형상에 주의 하면  $\frac{\partial L}{\partial \boldsymbol{X}}$과 $\frac{\partial L}{\partial \boldsymbol{W}}$은 이전과 같이 도출 가능.  
편향을 더할 때도 주의해야 함. 편향은 N개 Row에 Broadcasting하여 더해짐.  
아래 예로 편향의 계산을 확인 가능.  

In [10]:
X_dot_W = np.array( [[0, 0, 0], [10, 10, 10]] )
B = np.array( [1, 2, 3] )

print(X_dot_W, "\n")
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]] 

[[ 1  2  3]
 [11 12 13]]


순전파의 편향 덧셈은 각각의 데이터(1번째 데이터, 2번째 데이터, ...)에 더해짐.  
역전파 때는 형상을 맞춰야 하므로 데이터의 역전파 값이 편향의 원소에 모여야 함.  
코드로는 아래와 같음.  

In [11]:
dY = np.array( [[1, 2, 3], [4, 5, 6]] )
print(dY, "\n")

dB = np.sum(dY, axis = 0) # axis = 0은 row 기준.
print(dB)

[[1 2 3]
 [4 5 6]] 

[5 7 9]


Affine 계층 구현을 python으로 하면 아래와 같음.  
(common/layers.py의 구현은 데이터가 4차원인 경우도 고려한 것이라 아래의 구현과는 약간 차이가 존재.)

In [12]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

### 5.6.3 Softmax-with-Loss 계층  

* 소프트맥스 함수는 입력 값을 정규화하여 출력.  
    * 출력값의 합이 1이 되도록.  

손글씨 숫자 인식에서의 Softmax 계층의 출력은 **Frigure 5-28**과 같음.  

<center>**Figure 5-28 입력 이미지가 Affine 계층과 ReLU 계층을 통과하며 변환되고, 마지막 Softmax 계층에 의해서 10개의 입력이 정규화된다.  
이 그림에서는 숫자 '0'의 점수는 5.3이며, 이것이 Softmax 계층에 의해서 0.008(0.8%)로 변환된다. 또, '2'의 점수는 10.1에서 0.991(99.1%)로 변환된다.**</center>
![Figure_5-28](./images/Figure_5-28.png)  

**Figure 5-28**과 같이 Softmax 계층은 입력 밧을 정규화(출력의 합이 1이 되도록 변형)하여 출력.  
손글씨 숫자는 가짓수가 10개(10클래스 분류)이므로 Softmax 계층의 입력과 출력은 10개가 됨.  

* 신경망 추론에서는 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되니 Softmax 계층은 사용하지 않음.
* 반면, 신경망을 학습할때는 Softmax 계층이 필요.

소프트맥스 계층을 구현할 때, 손실 함수인 교차 엔트로피 오차도 포함하여 **Softmax-with-Loss 계층**이라는 이름으로 구현.  
**Figure 5-29**에서 Softmax-with-Loss 계층의 계산 그래프를 확인.  

<center>**Figure 5-29 Softmax-with-Loss 계층의 계산 그래프**</center>
![Figure_5-29](./images/Figure_5-29.png)  

Softmax-with-Loss 계층은 다소 복잡하나, **Figure 5-30**의 계산 그래프와 같이 간소화할 수 있음.  

<center>**Figure 5-30 '간소화한' Softmax-with-Loss 계층의 계산 그래프**</center>
![Figure_5-30](./images/Figure_5-30.png)  

예시)  
$t = (0, 1, 0)$이고 Softmax 계층이 $(0.3, 0.2, 0.5)$출력했다고 가정 하면, Softmax 계층의 역전파는 $(0.3, -0.8, 0.5)$로 커다란 오차를 전파.  
$t = (0, 1, 0)$이고 Softmax 계층이 $(0.01, 0.99, 0)$출력했다고 가정 하면, Softmax 계층의 역전파는 $(0.01, -0.01, 0)$로 작은 오차를 전파. 

Softmax-with-Loss 계층을 python으로 구현하면 아래와 같음.  

In [13]:
class SoftWithLoss:
    def __init_(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_enter(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout = 1):
        batch_size = self.shape[0]
        dx = (self.y - self.t) / batch_size # Batch size로 나눈 것에 주의. 
        
        return dx

## 5.7 오차역전파법 구현하기

### 5.7.1 신경망 학습의 전체 그림  

* **전제**
    * 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적용하도록 조정하는 과정을 **학습**이라고 함.
    * 신경망 학습은 아래 4단계로 수행.
* **1단계 - 미니배치**
    * 훈련 데이터 중 일부를 무작위로 가져옴. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것을 목표로 함.
* **2단계 - 기울기 산출**
    * 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함.
    * 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시.
* **3단계 - 매개변수 갱신**
    * 가중치 매개변수를 기울기 방향으로 아주 조금 갱신.
* **4단계 - 반복**
    * 1~3단계를 반복.

오차역전파법이 등장하는 단계는 두번째인 **기울기 산출**.  
앞 장에서 배운 수치 미분은 구현은 쉽지만 비효율적임.  
오차역전파법을 이용하면 느린 수치 미분과 달리 기울기를 효율적이고 빠르게 구할 수 있음.  

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

In [14]:
import sys, os
# sys.pasth.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # initial weigths
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # create layers                                                       # Important
        self.layers = OrderedDict()                                           # Important
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) # Important
        self.layers['Relu1'] = Relu()                                         # Important
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2']) # Important
        self.lastLayer = SoftmaxWithLoss()                                    # Important
        
    def predict(self, x):
        for layer in self.layers.values():                                    # Important
            x = layer.forward(x)                                              # Important
        
        return x
    
    # x : input data, t : true label
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1:
            t = np.argmax(t, axis = 1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    # x : input data, t : true label
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        # forward propagation
        self.loss(x, t)
        
        # backward propagation
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # result save
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

**신경망 계층을 OrderedDict에 보관하는 점이 중요**
OrderedDict는 순서가 있는 딕셔너리.  
**순서가 있는**이란 딕서너리에 추가한 순서를 기억한다는 것.  

* 순전파 때는 순서대로 각 계층의 forward() 메서드를 호출하면 되고, 역전파 때는 역순으로 backward() 메서드를 호출해주면 됨.  

### 5.7.3 오차역전파법으로 구한 기울기 검증하기

In [15]:
import sys, os
# sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoL1ayerNet

# load data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# calc abs error per weight
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

b2:1.19459991899e-10
W2:7.16269371939e-13
b1:7.71072048645e-13
W1:2.08732567469e-13


### 5.7.4 오차역전파법을 사용한 학습 구현하기  

수치 미분의 이점은 구현하기 쉽고, 버그가 존재하기 힘드므로 실수하기 쉬운 오차역전파법 구현 검증에 사용.  

두 방식으로 기울기가 일치함(거의 같음)을 확인하는 작업을 **기울기 확인(gradient check)**라고 함.  
두 방식의 오차가 아주 작으면 정상.  

In [1]:
import sys, os
#sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoL1ayerNet

# load data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # calculate gradient with backpropagation
    grad = network.gradient(x_batch, t_batch)
    
    # 
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("Train Accuracy : ", round(train_acc, 4), "Test Accuracy : ", test_acc)


NameError: name 'TwoLayerNet' is not defined

In [2]:
import numpy as np


$$
\frac{b}{a}
$$